In [3]:
from datasets import load_dataset

openbookqa = load_dataset("openbookqa", "additional")

Generating train split:   0%|          | 0/4957 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset openbookqa downloaded and prepared to /home/diwank/.cache/huggingface/datasets/openbookqa/additional/1.0.1/f338ccacfbc86fb8c2de3aa1c06d2ce686933de3bca284dba97d32592c52b33f. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
openbookqa_subset = openbookqa.filter(lambda row: (
    row["humanScore"] >= 1.0
    and row["clarity"] >= 1.5
    and row["question_stem"].strip().endswith('?')
))

Loading cached processed dataset at /home/diwank/.cache/huggingface/datasets/openbookqa/additional/1.0.1/f338ccacfbc86fb8c2de3aa1c06d2ce686933de3bca284dba97d32592c52b33f/cache-c804c802d4277e46.arrow
Loading cached processed dataset at /home/diwank/.cache/huggingface/datasets/openbookqa/additional/1.0.1/f338ccacfbc86fb8c2de3aa1c06d2ce686933de3bca284dba97d32592c52b33f/cache-18f033fba0983f6b.arrow
Loading cached processed dataset at /home/diwank/.cache/huggingface/datasets/openbookqa/additional/1.0.1/f338ccacfbc86fb8c2de3aa1c06d2ce686933de3bca284dba97d32592c52b33f/cache-a8cb5bdc97d31241.arrow


In [ ]:
make_chatml = lambda name, role, content: dict(
    name=name, role=role, content=content,
)

system = lambda name, content: make_chatml(
    role="system",
    name=name,
    content=content,
)

situation = lambda content: system(name="situation", content=content)
thought = lambda content: system(name="thought", content=content)
information = lambda content: system(name="information", content=content)
me = lambda content, name=None: make_chatml(
    role="assistant",
    content=content,
    name=name,
)

person = lambda content, name=None: make_chatml(
    role="user",
    content=content,
    name=name,
)

def to_chatml(row):
    question = row["question_stem"]
    choices = row["choices"]
    fact = row["fact1"]
    answer_key = row["answerKey"]

    choices = "\n".join([
        f"{label}) {text}"
        for label, text
        in zip(choices["label"], choices["text"])
    ])

    siutation_text = (
        "A person is talking to an intelligent AI assistant."
        " The person is quizzing the AI assistant on miscellaneous questions"
        " in a multiple choice format."
        " The assistant first thinks carefully about the question and lays down its reasoning."
        " Then answers the question with the correct choice key and the right answer."
    )
    
    mcq = f"Question: {question}\nChoices:\n{choices}"
    thought_text
    answer_text 
    
    chatml = [
        situation(),
        person(),
        thought(),
        me(),
    ]
    
    return dict(chatml=chatml)

openbookqa_chatml = openbookqa_subset.map(
    to_chatml
).remove_columns(openbookqa_subset.column_names)